GitHub  
https://github.com/PaddlePaddle/PaddleOCR  


## 実行前準備
「ランタイム」→「ランタイムのタイプを変更」→「ハードウェアアクセラレータ」をGPUに変更

## 実行方法
「ランタイム」→「すべてのセルを実行」を選択

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# PaddleOCRのソースコードをGitHubから取得

In [ ]:
%cd /content/
!git clone https://github.com/PaddlePaddle/PaddleOCR.git

# 関連するライブラリのインストール

In [ ]:
%cd /content/PaddleOCR
!pip install -r requirements.txt

# PaddlePaddleのインストール

In [ ]:
!python3 -m pip install paddlepaddle-gpu -i https://mirror.baidu.com/pypi/simple

# PaddlePaddleのインストール確認
PaddlePaddleのバージョンと認識しているGPUの数を出力

In [ ]:
!python3 -c "import paddle; print(paddle.__version__)"
!python3 -c "import paddle; paddle.utils.run_check()"

# テスト用画像のダウンロード

In [ ]:
%cd /content/PaddleOCR

!wget https://paddleocr.bj.bcebos.com/dygraph_v2.1/ppocr_img.zip
!unzip ppocr_img.zip > /dev/null

# 学習済みモデルのダウンロード

In [ ]:
%cd /content/PaddleOCR
!mkdir pretrain_models
%cd /content/PaddleOCR/pretrain_models

!wget https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_server_v2.0_det_infer.tar
!tar -xvf ch_ppocr_server_v2.0_det_infer.tar

# 文字検出

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw

In [ ]:
%cd /content/PaddleOCR
!python3 tools/infer/predict_det.py \
  --det_algorithm="DB" \
  --det_model_dir="./pretrain_models/ch_ppocr_server_v2.0_det_infer" \
  --image_dir="./ppocr_img/imgs_en/254.jpg" \
  --draw_img_save_dir="./inference_results" \
  --use_gpu=True

# 検出結果の表示

In [ ]:
%cd /content/PaddleOCR
result_path = "./inference_results/det_res_254.jpg"

# 画像の表示
image = Image.open(result_path).convert("RGB")
plt.figure(figsize=(6, 6))
plt.imshow(image)

# 文字検出のトレーニング

## データ準備

In [ ]:
%cd /content/PaddleOCR

# 登録必須
# 学習データダウンロード
!wget --no-check-certificate https://rrc.cvc.uab.es/downloads/ch4_training_images.zip
!wget --no-check-certificate https://rrc.cvc.uab.es/downloads/ch4_test_images.zip
!unzip ch4_training_images.zip -d ./icdar_c4_train_imgs > /dev/null
!unzip ch4_test_images.zip -d ./ch4_test_images > /dev/null

!wget https://paddleocr.bj.bcebos.com/dataset/train_icdar2015_label.txt
!wget https://paddleocr.bj.bcebos.com/dataset/test_icdar2015_label.txt

## 学習済みモデルのダウンロード

In [ ]:
%cd /content/PaddleOCR/pretrain_models

!wget https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_server_v2.0_det_train.tar
!tar -xvf ch_ppocr_server_v2.0_det_train.tar

## model保存用ディレクトリ作成

In [ ]:
!mkdir -p /content/drive/MyDrive/Colab \Notebooks/output/ch_db_res18/

# トレーニング

In [ ]:
%cd /content/PaddleOCR

!python3 tools/train.py -c configs/det/det_res18_db_v2.0.yml \
  -o Global.pretrained_model=./pretrain_models/ch_ppocr_server_v2.0_det_train/best_accuracy \
  Global.save_model_dir="/content/drive/MyDrive/Colab Notebooks/output/ch_db_res18/" \
  Global.save_res_path="/content/drive/MyDrive/Colab Notebooks/output/ch_db_res18/" \
  Train.dataset.data_dir=./ \
  Train.dataset.label_file_list=[./train_icdar2015_label.txt] \
  Eval.dataset.data_dir=./ \
  Eval.dataset.label_file_list=[./test_icdar2015_label.txt] \